Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

In [2]:
#pickle_file = 'notMNIST_u.pickle'  ## deduped
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (18724, 28, 28, 1) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with 2 convolutional layers, followed by 1 fully connected layer. 
-----------
Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [17]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  ## conv layers
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) ## feat map 1 -> 16
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1)) ## feat map 16 -> 16
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  ## fc layer
  ## 2 conv's reduced map size to 1/4 img x 1/4 img x depth 16
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size / 4 * image_size / 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  ## readout layer
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))  ## num hidden nodes -> num labels
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()  ## batch, height, width, depth
    print shape
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])

    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 7, 7, 16]
[6000, 7, 7, 16]
[18238, 7, 7, 16]


** Run it **

In [18]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized\n==============="
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.2f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.2f%%" % accuracy(valid_prediction.eval(), valid_labels)
      print "---------------------------"
  print "Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 3.1705
Minibatch accuracy: 12.50%
Validation accuracy: 9.53%
---------------------------
Minibatch loss at step 200 : 0.888526
Minibatch accuracy: 75.00%
Validation accuracy: 74.97%
---------------------------
Minibatch loss at step 400 : 0.471137
Minibatch accuracy: 87.50%
Validation accuracy: 79.08%
---------------------------
Minibatch loss at step 600 : 0.567177
Minibatch accuracy: 81.25%
Validation accuracy: 80.65%
---------------------------
Minibatch loss at step 800 : 1.26522
Minibatch accuracy: 68.75%
Validation accuracy: 81.05%
---------------------------
Minibatch loss at step 1000 : 0.696821
Minibatch accuracy: 87.50%
Validation accuracy: 83.33%
---------------------------
Minibatch loss at step 1200 : 0.260978
Minibatch accuracy: 87.50%
Validation accuracy: 83.23%
---------------------------
Minibatch loss at step 1400 : 0.832862
Minibatch accuracy: 81.25%
Validation accuracy: 84.18%
---------------------------
Minibatch loss at step 

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides with a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

** Two 4x4 conv w/ max pooling (16@14x14, 32@7x7), fc 128 **
> steps 6001, decay (.05, 100, .96), Test accuracy: **94.08**%<br>
> steps 12499, decay (.05, 200, .96), Test accuracy: **94.89**%<br>
> steps 12499, decay (.04, 100, .96), Test accuracy: **93.89**%<br>
> steps 6001, decay (.04, 200, .96), Test accuracy: **94.22**%<br>
> steps 6001, decay (.05, 200, .96), Test accuracy: **94.45**%<br>

    12001 0.00431673
    Minibatch loss at step 12000 : 0.0330466
    Minibatch accuracy: 100.00%
    Validation accuracy: 89.47%
    Test accuracy: 94.89%

** Two 3x3 conv w/ max pooling (6@14x14, 16@7x7), fc 256 **
> steps 12499, decay (.05, 200, .96), Test accuracy: **94.46**%<br>

    12001 0.00431673
    Minibatch loss at step 12000 : 0.0192785
    Minibatch accuracy: 100.00%
    Validation accuracy: 88.44%
    Test accuracy: 94.46%

** Two 5x5 conv w/ max pooling (6@14x14, 16@7x7), fc 512 **
> steps 6001, decay (.05, 200, .96), Test accuracy: **94.34**%<br>

    6001 0.0146899
    Minibatch loss at step 6000 : 0.630632
    Minibatch accuracy: 68.75%
    Validation accuracy: 88.40%

** Two 5x5 conv w/ max pooling (32@14x14, 64@5x5), fc 1024 **

    Minibatch loss at step 6000 : 0.189193
    Minibatch accuracy: 93.75%
    Validation accuracy: 89.72%
    Test accuracy: 95.25%

    Minibatch loss at step 12000 : 0.159131
    Minibatch accuracy: 93.75%
    Validation accuracy: 90.52%
    Test accuracy: 95.85%

    Minibatch loss at step 18000 : 0.353096
    Minibatch accuracy: 87.50%
    Validation accuracy: 91.04%
    Test accuracy: 96.20%

---

** Helper functions **

In [5]:
## Weight Initialization
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape, constant=0.1):
  initial = tf.constant(constant, shape=shape)
  return tf.Variable(initial)

In [6]:
## Convolution and Pooling
def conv2d(x, W, padding='SAME'):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding=padding)

def max_pool_2x2(x, padding='SAME'):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding=padding)

** Graph w/ max pooling**

In [14]:
batch_size = 32
patch_size = 5
depth = 32
depth2 = 64
num_hidden = 1024

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = weight_variable([patch_size, patch_size, num_channels, depth])
  layer1_biases = bias_variable([depth], 0.0)
  layer2_weights = weight_variable([patch_size, patch_size, depth, depth2])
  layer2_biases = bias_variable([depth2])
  ## fc layer
  layer3_weights = weight_variable([(image_size/4 - 2)**2 * depth2, num_hidden])
  layer3_biases = bias_variable([num_hidden])
  ## readout layer
  layer4_weights = weight_variable([num_hidden, num_labels])
  layer4_biases = bias_variable([num_labels])

  ## Dropout
  ##keep_prob = tf.placeholder(tf.float32)
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden = max_pool_2x2(hidden)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer2_biases)
    hidden = max_pool_2x2(hidden)
    shape = hidden.get_shape().as_list()
    print shape
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    ##hidden = tf.nn.dropout(hidden, keep_prob)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  #global_step = tf.Variable(0, trainable=False) # count the number of steps taken.
  #learning_rate = tf.train.exponential_decay(0.05, global_step, 200, 0.96, staircase=False)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  optimizer = tf.train.AdagradOptimizer(0.02).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[32, 5, 5, 64]
[10000, 5, 5, 64]
[18724, 5, 5, 64]


** Run w/ max pooling **

In [15]:
num_steps = 18001
def step_eval():
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      #print global_step.eval(), learning_rate.eval()
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.2f%%" % accuracy(predictions, batch_labels)
      accuracy_valid = accuracy(valid_prediction.eval(), valid_labels)
      print "Validation accuracy: %.2f%%" % accuracy_valid
      print "-" * 20
      if accuracy_valid > 92.:
        print "Halted!"
        return
    if (step % 3000 == 0):
      print "Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels)
      print "=" * 20

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized\n", "="*16
  step_eval()
  print "Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 4.71037
Minibatch accuracy: 18.75%
Validation accuracy: 13.19%
--------------------
Test accuracy: 13.65%
Minibatch loss at step 1000 : 0.744925
Minibatch accuracy: 75.00%
Validation accuracy: 85.84%
--------------------
Minibatch loss at step 2000 : 1.02551
Minibatch accuracy: 75.00%
Validation accuracy: 87.64%
--------------------
Minibatch loss at step 3000 : 0.414678
Minibatch accuracy: 81.25%
Validation accuracy: 88.62%
--------------------
Test accuracy: 94.21%
Minibatch loss at step 4000 : 0.272227
Minibatch accuracy: 93.75%
Validation accuracy: 89.20%
--------------------
Minibatch loss at step 5000 : 0.397788
Minibatch accuracy: 87.50%
Validation accuracy: 89.41%
--------------------
Minibatch loss at step 6000 : 0.31275
Minibatch accuracy: 90.62%
Validation accuracy: 89.68%
--------------------
Test accuracy: 95.24%
Minibatch loss at step 7000 : 0.58002
Minibatch accuracy: 84.38%
Validation accuracy: 89.80%
--------------------
Minibatch

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

http://cs231n.github.io/convolutional-networks/#architectures

http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf

INPUT 28x28 -> <br>
C1 6@28x28 -> S2 6@14x14 -> <br>
C3 16@10x10 -> S4 16@5x5 -> <br>
C5 layer 120 -> F6 layer 84 -> <br>
OUPUT 10

**original data**

    6001 0.00187146
    Minibatch loss at step 6000 : 0.6133
    Minibatch accuracy: 81.25%
    Validation accuracy: 87.17%
    Test accuracy: 93.48%
    
    Minibatch loss at step 12000 : 0.0926194
    Minibatch accuracy: 96.88%
    Validation accuracy: 88.70%
    Test accuracy: 94.14%
    
    Minibatch loss at step 30000 : 0.396998
    Minibatch accuracy: 87.50%
    Validation accuracy: 89.30%
    Test accuracy: 94.92%
    
    Minibatch loss at step 42000 : 0.16477
    Minibatch accuracy: 96.88%
    Validation accuracy: 89.48%
    Test accuracy: 95.18%
    
    Minibatch loss at step 60000 : 0.488225
    Minibatch accuracy: 90.62%
    Validation accuracy: 89.90%
    Test accuracy: 95.40%
    
**original data, depth 16, 32**

    Minibatch loss at step 6000 : 0.680322
    Minibatch accuracy: 75.00%
    Validation accuracy: 86.73%
    Test accuracy: 92.86%
    
    Minibatch loss at step 12000 : 0.0486726
    Minibatch accuracy: 100.00%
    Validation accuracy: 88.26%
    Test accuracy: 94.24%    
    
    Minibatch loss at step 30000 : 0.0725141
    Minibatch accuracy: 100.00%
    Validation accuracy: 89.70%
    Test accuracy: 95.07%
    
**deduped data**

    Minibatch loss at step 6000 : 0.48905
    Minibatch accuracy: 87.50%
    Validation accuracy: 88.57%
    Test accuracy: 94.39%
    
    Minibatch loss at step 60000 : 0.265826
    Minibatch accuracy: 93.75%
    Validation accuracy: 90.95%
    Test accuracy: 96.22%

---

** LeNet-5 ** 

In [10]:
batch_size = 32
patch_size = 5
depth = 6
depth2 = 16
num_hidden = 120
num_hidden2 = 84

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = weight_variable([patch_size, patch_size, num_channels, depth])
  layer1_biases = bias_variable([depth])
  layer2_weights = weight_variable([patch_size, patch_size, depth, depth2])
  layer2_biases = bias_variable([depth2])
  ## fc layer
  layer3_weights = weight_variable([((image_size/2 - patch_size+1)/2)**2 * depth2, num_hidden])
  layer3_biases = bias_variable([num_hidden])
  ## fc2 layer
  layer4_weights = weight_variable([num_hidden, num_hidden2])
  layer4_biases = bias_variable([num_hidden2])
  ## readout layer
  layer5_weights = weight_variable([num_hidden2, num_labels])
  layer5_biases = bias_variable([num_labels])

  ## Dropout
  #keep_prob = tf.placeholder(tf.float32)
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden = max_pool_2x2(hidden)
    conv = conv2d(hidden, layer2_weights, padding='VALID')
    hidden = tf.nn.relu(conv + layer2_biases)
    hidden = max_pool_2x2(hidden)
    shape = hidden.get_shape().as_list()
    print shape
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print reshape.get_shape().as_list()
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    hidden = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)
    #hidden = tf.nn.dropout(hidden, keep_prob)
    return tf.matmul(hidden, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  #global_step = tf.Variable(0, trainable=False) # count the number of steps taken.
  #learning_rate = tf.train.exponential_decay(0.05, global_step, 200, 0.96, staircase=False)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  optimizer = tf.train.AdagradOptimizer(0.01).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[32, 5, 5, 16]
[32, 400]
[10000, 5, 5, 16]
[10000, 400]
[18724, 5, 5, 16]
[18724, 400]


** Run it, LeNet-5**

In [11]:
num_steps = 62501
def step_eval():
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 3000 == 0):
      #print global_step.eval(), learning_rate.eval()
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.2f%%" % accuracy(predictions, batch_labels)
      accuracy_valid = accuracy(valid_prediction.eval(), valid_labels)
      print "Validation accuracy: %.2f%%" % accuracy_valid
      print "---------------------------"
      if accuracy_valid > 92.:
        print "Halted!"
        return
    if (step % 12000 == 0):
      print "Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels)
      print "=" * 20

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized\n==============="
  step_eval()
  print "Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 2.2556
Minibatch accuracy: 12.50%
Validation accuracy: 10.27%
---------------------------
Test accuracy: 10.26%
Minibatch loss at step 3000 : 0.58522
Minibatch accuracy: 84.38%
Validation accuracy: 85.97%
---------------------------
Minibatch loss at step 6000 : 0.307808
Minibatch accuracy: 96.88%
Validation accuracy: 87.38%
---------------------------
Test accuracy: 93.32%
Minibatch loss at step 9000 : 0.274287
Minibatch accuracy: 90.62%
Validation accuracy: 88.11%
---------------------------
Minibatch loss at step 12000 : 0.0926194
Minibatch accuracy: 96.88%
Validation accuracy: 88.70%
---------------------------
Test accuracy: 94.14%
Minibatch loss at step 15000 : 0.365732
Minibatch accuracy: 90.62%
Validation accuracy: 88.84%
---------------------------
Minibatch loss at step 18000 : 0.481385
Minibatch accuracy: 84.38%
Validation accuracy: 88.89%
---------------------------
Test accuracy: 94.48%
Minibatch loss at step 21000 : 0.24859
Minibatch

---
MNIST data
---------
https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html#build-a-multilayer-convolutional-network

---

In [22]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
